 Replicating Kirkland (2021)

Answer:
1. RDD Types:

There are two primary types of RDD:

Sharp RDD: Occurs when the assignment to treatment (business executive candidate) is deterministic based on the cutoff (vote share margin of 0). In this case, individuals just on either side of the cutoff are perfectly comparable due to identical eligibility except for the treatment assignment.
Fuzzy RDD: The assignment probability changes smoothly around the cutoff. Individuals near the cutoff may have different characteristics than those further away, making comparisons less clear-cut.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.regression.linear_model import OLS

# Load data (assuming csv format)
data = pd.read_csv("kirkland_data.csv")

# Define variables
running_var = "exec_margin"
outcome_var1 = "pc_roads_lead2"
outcome_var2 = "pc_housing_lead2"
cutoff = 0


Question 2:
Check the distribution of running variable, and see if there is anything suspicious.

ANSWER:

2. Checking Running Variable Distribution:

We can analyze the distribution of exec_margin using:

Histograms: Visually assess the density around the cutoff (0).
Kernel density plots: Show a smoother estimate of the probability distribution.
Summary statistics: Analyze mean, median, and standard deviation to see if there are any major imbalances near the cutoff.
Look for signs of:

Bunching: If individuals "game" the cutoff by strategically positioning themselves just above or below it.
Discontinuity: Check if the density drops sharply at the cutoff, indicating a possible sharp RDD.

In [ ]:
import seaborn as sns

# Histogram
data[running_var].hist()
plt.xlabel(running_var)
plt.ylabel("Frequency")
plt.title("Distribution of Running Variable")
plt.show()

# Kernel density plot
plt.figure(figsize=(10, 6))
sns.kdeplot(data[running_var], shade=True, label=f"Density of {running_var}")
plt.xlabel(running_var)
plt.ylabel("Density")
plt.axvline(cutoff, color="red", linestyle="--", label="Cutoff")
plt.legend()
plt.show()

# Try converting running_var to numeric (handle errors)
try:
    data[running_var] = pd.to_numeric(data[running_var], errors='coerce')
except:
    print(f"Warning: Unable to convert {running_var} to numeric for all values.")

# Calculate summary statistics (after potential conversion)
print(data.groupby(data[running_var] < cutoff)[running_var].describe())


Question 3:

Visualize the treatment effect by plotting the running variable as x and outcome variable as y, with the regression or some smoothing lines in both sides
of the cutpoint.

Answer:

3. Visualization:

We can plot the following:

Scatter plot: exec_margin on x-axis and roads_lead2/parks_lead2 on y-axis with different colours for values above and below the cutoff.
Local linear regression lines: Overlaid on the scatter plot, showing the estimated treatment effect on either side of the cutoff.
Confidence intervals: Around the regression lines to assess the precision of the estimates.

In [ ]:
# Scatter plot with regression lines
plt.figure(figsize=(10, 6))

plt.scatter(data[running_var], data[outcome_var1], c=data[running_var] < cutoff, cmap="coolwarm")
model = OLS(data[outcome_var1], pd.concat([data[running_var], data[running_var] * (data[running_var] > cutoff)], axis=1))
rd_lm = model.fit()
# plt.plot(data[running_var], rd_lm.predict(data[["running_var", "new_intersection"]]), color="blue")

plt.xlabel(running_var)
plt.ylabel(outcome_var1)
plt.title(f"Treatment Effect on {outcome_var1}")
plt.legend()
plt.show()

# Similar plot for the second outcome variable
plt.figure(figsize=(10, 6))

plt.scatter(data[running_var], data[outcome_var2], c=data[running_var] < cutoff, cmap="coolwarm")
model = OLS(data[outcome_var2], pd.concat([data[running_var], data[running_var] * (data[running_var] > cutoff)], axis=1))
rd_lm = model.fit()
# plt.plot(data[running_var], rd_lm.predict(data[["running_var", "running_var * (running_var > cutoff)"]]), color="blue")

plt.xlabel(running_var)
plt.ylabel(outcome_var2)
plt.title(f"Treatment Effect on {outcome_var2}")
plt.legend()
plt.show()


Question 4:

Estimate the size of the Local Average Treatment Effect (LATE). Try both
parametric and nonparametric estimates, and see the differences.

Answer:

4. Local Average Treatment Effect (LATE):

We can estimate the LATE using both parametric and nonparametric methods:

Parametric: Use rd function from rdtools package with a regression model (e.g., linear regression).
Nonparametric: Estimate the average outcome for treated and control groups near the cutoff and compute the difference.
Compare the LATE estimates from both methods and explore why they might differ.

Additional Notes:

Consider performing robustness checks like falsification tests and placebo tests.
Interpret the results cautiously, acknowledging the limitations of RDD.
Document your analysis steps and findings clearly.

In [ ]:
import statsmodels.api as sm
# 4. Estimate the size of the Local Average Treatment Effect (LATE)
# Parametric estimate
parametric_result_roads = sm.OLS(data['pc_roads_lead2'], sm.add_constant(data['exec_margin'])).fit()
parametric_result_parks = sm.OLS(data['pc_housing_lead2'], sm.add_constant(data['exec_margin'])).fit()

print(f'Parametric LATE estimate for pc_roads_lead2: {parametric_result_roads.params["exec_margin"]}')
print(f'Parametric LATE estimate for pc_housing_lead2: {parametric_result_parks.params["exec_margin"]}')

Parametric LATE estimate for pc_roads_lead2: nan
Parametric LATE estimate for pc_housing_lead2: nan


3 IPP Dataset: Data preparation and description

Question 1:

1. Load the dataset in R. Show the code. Print the dimensions and class of the
object containing the data in your workspace.

In [ ]:
import pandas as pd

# Corrected file path
path_to_dataset = "C:/Users/PMLS/Desktop/kik/"
file_name = "IPP_wide_25_09_22.csv"

# Read the dataset into a pandas DataFrame
ipp_data = pd.read_csv(f"{path_to_dataset}{file_name}")

# Print dimensions and class of the DataFrame
print(ipp_data.shape)


Explanation:

The pandas library is used to read and manipulate the dataset. It's a powerful library for data analysis in Python.
pd.read_csv is used to read the CSV file into a DataFrame.
ipp_data.shape prints the dimensions of the DataFrame (number of rows and columns).
ipp_data.dtypes prints the data types of each column in the DataFrame.

Question 2:

Using R, create up to two plots that summarise interesting aspects of the data,
perhaps (but not necessarily) with a view to the analysis in Section 4. Describe
the plots and their findings briefly

Task 2: Create up to two plots summarizing interesting aspects of the data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Plot 1: Distribution of Age
plt.figure(figsize=(8, 6))
sns.histplot(ipp_data['w1_dg_Meretz'], bins=20, kde=True, color='blue')
plt.title('Distribution of Age in IPP Dataset')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

# Corrected column names
plt.figure(figsize=(10, 6))
sns.countplot(x='w3_dg_Yahadut-Hatorah', hue='w3_dg_Yahadut-Hatorah', data=ipp_data, palette='Set2')
plt.title('Political Attitudes by Gender')
plt.xlabel('Political Attitude')
plt.show()


Explanation:

The matplotlib.pyplot and seaborn libraries are used for plotting.
For the first plot, sns.histplot is used to create a histogram of the 'age' variable with kernel density estimation (KDE). This visualizes the distribution of ages in the dataset.
For the second plot, sns.countplot is used to create a bar plot of 'political_attitude' by 'gender'. This visualizes the count of each political attitude category for each gender.
plt.show() is used to display the plots.

4 IPP Dataset: Analysis

Research Question:
"To what extent do demographic factors (such as age, gender, and education level) influence political attitudes in the Israeli population, as captured by the IPP dataset?"

Introduction
1.1 Overview of the IPP Dataset
The Israel Polarization Panel (IPP) dataset, as curated by Gidron et al. (2022), constitutes a rich collection of survey data gathered across ten waves spanning from 2019 to 2021. This dataset encapsulates a comprehensive snapshot of the Israeli population's sentiments, encompassing demographics, political attitudes, voting intentions, and an array of other traits and opinions. The availability of such a nuanced dataset provides an unprecedented opportunity for political scientists to delve into the intricacies of Israeli society, unraveling patterns and relationships that underlie the interplay between demographic factors and political attitudes.

1.2 Significance of Demographic Factors in Political Attitudes
Understanding the dynamics between demographic factors and political attitudes is imperative for comprehending the nuanced landscape of a society. Demographic characteristics such as age, gender, and education level often serve as crucial determinants shaping individuals' perspectives on political matters. These factors not only influence the formation of political ideologies but also play a pivotal role in electoral choices and policy preferences. Investigating these relationships provides a nuanced lens through which to interpret societal dynamics, aiding in the development of informed political strategies and policies.

1.3 Research Question
In light of the wealth of information encapsulated within the IPP dataset, our research seeks to address the following fundamental question:

"To what extent do demographic factors, including age, gender, and education level, impact political attitudes in the Israeli population, as captured by the IPP dataset?"

This research question serves as the guiding compass for our analysis, steering us towards a comprehensive exploration of the intricate connections between demographics and political sentiments within the Israeli context.

2. Literature Review
2.1 Relationship Between Demographics and Political Attitudes
Understanding the interplay between demographic factors and political attitudes has been a central focus in political science research. A review of existing literature reveals a wealth of studies exploring how variables such as age, gender, and education level contribute to shaping individuals' political perspectives.

2.2 Key Findings from Relevant Studies
2.2.1 Age and Political Attitudes
Numerous studies have delved into the impact of age on political attitudes. [Author et al. (Year)] found that younger individuals tend to be more progressive in their political views, while older cohorts may exhibit more conservative tendencies. However, the relationship between age and political attitudes is multifaceted, with variations observed across different regions and political climates.

2.2.2 Gender and Political Attitudes
The role of gender in shaping political attitudes has also garnered significant attention. [Author et al. (Year)] identified distinct gender-based patterns, indicating that women may lean towards certain political ideologies compared to their male counterparts. The gender gap in political preferences has implications for electoral outcomes and policy considerations.

2.2.3 Education Level and Political Attitudes
Education emerges as a key determinant of political attitudes, as explored by [Author et al. (Year)]. Higher education levels are often associated with increased political engagement and a propensity for liberal ideologies. However, conflicting findings exist, necessitating a nuanced examination of how education interacts with other demographic factors.

2.3 Gaps and Inconsistencies in the Literature
Despite the wealth of research on demographics and political attitudes, certain gaps and inconsistencies persist. Some studies may focus predominantly on one demographic factor, overlooking potential interactions or confounding variables. Additionally, regional variations and changing socio-political landscapes introduce complexities that warrant further investigation. Our research aims to contribute to addressing these gaps by leveraging the comprehensive IPP dataset to provide a nuanced understanding of the Israeli context.



3. Methods
3.1 Overview of the IPP Dataset
The Israel Polarization Panel (IPP) dataset serves as the foundation for our analysis. Compiled by Gidron et al. (2022), this dataset spans ten waves from 2019 to 2021, capturing a diverse array of survey responses from the Israeli population. The dataset encapsulates demographics, political attitudes, voting intentions, and various other traits, offering a comprehensive lens into the intricate fabric of Israeli societal dynamics.

3.2 Variables Used in the Analysis
Our analysis centers around several key variables extracted from the IPP dataset, chosen to address the overarching research question regarding the relationship between demographic factors and political attitudes. The variables under scrutiny include:

Age: The age of respondents, providing insight into generational perspectives.

Gender: A categorical variable capturing the gender identity of respondents.

Education Level: An indicator of the educational background of respondents, ranging from basic education to advanced degrees.

Political Attitudes: A composite measure gauging respondents' political inclinations, encompassing factors such as ideology and party affiliation.

3.3 Justification for Variable Selection
The selection of these variables aligns with the core research question, aiming to unravel the nuanced connections between demographic factors and political attitudes within the Israeli population. Age, gender, and education level are recognized determinants in political science literature, and their inclusion ensures a comprehensive exploration of potential influences on political perspectives. Political attitudes, as a composite measure, enable a holistic assessment, considering the multifaceted nature of political ideologies.

3.4 Preprocessing Steps
Handling Missing Data
Prior to analysis, we conducted thorough checks for missing data. Any missing values were addressed through appropriate imputation techniques, ensuring the integrity of the dataset.

Encoding Categorical Variables
Categorical variables, such as gender, were encoded to facilitate their incorporation into statistical models. This involved assigning numerical representations to categorical labels, maintaining the interpretability of results.

These preprocessing steps were crucial in preparing a robust dataset for analysis, mitigating the impact of missing data and ensuring compatibility with the selected analytical methods.

4. Analysis/Findings
4.1 Descriptive Statistics
Summary Statistics
To gain a preliminary understanding of our variables of interest, we present summary statistics for both demographic variables and political attitudes.

In [ ]:
# Display summary statistics
ipp_data[['age', 'gender', 'education_level', 'political_attitudes']].describe()

Visualization of Key Variables
The distribution of key variables is visually depicted through histograms, box plots, and other relevant visualizations.

In [ ]:
# Visualize the distribution of age
plt.figure(figsize=(8, 6))
sns.histplot(ipp_data['age'], bins=20, kde=True, color='blue')
plt.title('Distribution of Age in IPP Dataset')
plt.xlabel('Age')
plt.show()

4.2 Bivariate Analysis
Relationship Between Demographic Factors and Political Attitudes
We delve into the bivariate relationships between demographic factors and political attitudes, employing scatter plots and box plots for visual insights.

In [ ]:
# Scatter plot: Age vs. Political Attitudes
plt.figure(figsize=(10, 6))
sns.scatterplot(x='age', y='political_attitudes', data=ipp_data, color='green')
plt.title('Age vs. Political Attitudes')
plt.xlabel('Age')
plt.ylabel('Political Attitudes')
plt.show()

4.3 Multivariate Analysis
Regression Analysis
To disentangle the combined effects of demographic variables on political attitudes, we perform a multivariate regression analysis.

In [ ]:
# Fit a linear regression model
model = smf.ols('political_attitudes ~ age + gender + education_level', data=ipp_data).fit()
print(model.summary())

.4 Additional Analyses
Subgroup Analyses
We explore subgroup analyses to discern potential variations in the relationship between age, gender, and political attitudes.

In [ ]:
# Age group analysis
ipp_data['age_group'] = pd.cut(ipp_data['age'], bins=[18, 30, 50, 70, 100], labels=['18-30', '31-50', '51-70', '71-100'])



4.4 Additional Analyses
Subgroup Analyses
We explore subgroup analyses to discern potential variations in the relationship between age, gender, and political attitudes.

In [ ]:
# Conduct subgroup analysis
subgroup_analysis = smf.ols('political_attitudes ~ age_group + gender', data=ipp_data).fit()
print(subgroup_analysis.summary())

Sensitivity Analyses
Incorporate sensitivity analyses to validate the robustness of findings, considering alternative model specifications or subsets.



# 5. Conclusions

## 5.1 Summary of Key Findings

The analysis of the IPP dataset has provided valuable insights into the relationship between demographic factors (age, gender, education level) and political attitudes. Key findings include [highlight the main discoveries and patterns observed in the data].

## 5.2 Implications and Limitations

The implications of these results are significant for understanding the complex interplay between demographics and political attitudes. However, it is essential to acknowledge certain limitations. [Discuss any constraints or potential biases in the dataset or methodology.]

## 5.3 Highlighting Limitations

While this analysis contributes valuable insights, it is essential to recognize certain limitations:

- [Limitation 1: Explain the limitation and its impact on the results.]
- [Limitation 2: Discuss how this may affect the generalizability of findings.]
- [Limitation 3: Address any other potential challenges faced during the analysis.]

# 6. Equations and Python Code

## 6.1 Equations

The following equations were used in the analysis:

### Equation 1

\[ Y = \beta_0 + \beta_1X_1 + \beta_2X_2 + \epsilon \]

### Equation 2

\[ Z = \alpha_0 + \alpha_1W_1 + \alpha_2W_2 + \mu \]

## 6.2 Python Code Snippets

### Loading Data

```python
import pandas as pd

# Load the dataset into Python
ipp_data = pd.read_csv("path/to/dataset.csv")


Descriptive Statistics


In [ ]:
# Display summary statistics
print(ipp_data[['age', 'gender', 'education_level', 'political_attitudes']].describe())


Regression Analysis


In [ ]:
import statsmodels.api as sm

# Fit a linear regression model
X = ipp_data[['age', 'gender', 'education_level']]
y = ipp_data['political_attitudes']
model = sm.OLS(y, sm.add_constant(X)).fit()
print(model.summary())
